In [23]:
from osgeo import gdal
from osgeo import ogr
from osgeo import osr
from osgeo import gdal_array
from osgeo import gdalconst
import os
import json
import folium



def geomToGeoJSON(in_geom, name, simplify_tolerance= None, in_ref = None, out_ref = None,outPath=None):
    '''
    Matry's Function! Converts Geometry to GeoJSON
    '''
    if not simplify_tolerance is None:
        in_geom = in_geom.Simplify(simplify_tolerance)
    transform = osr.CoordinateTransformation(in_ref, out_ref)      
    
    #don't want to affect original geometry
    transform_geom = in_geom.Clone()      

    #trasnsform geometry from whatever the local projection is to wgs84
    if in_ref is None or out_ref is None:
        pass
    else:
        transform_geom.Transform(transform)
        
    json_text = transform_geom.ExportToJson()
    #add some attributes

    geom_json = json.loads(json_text)
    #get area in local units

    area = in_geom.GetArea()
    
    geojson_dict = {

        "type": "Feature",

        "geometry": geom_json,

        "properties": {

            "area": area

        }

    }
    geojson = json.dumps(geojson_dict)
    if not outPath is None:

        f = open('./' + name + '.geojson','w')

        f.write(geojson)

        f.close()

        print('Exported geojson:', name)       

    return geojson

class Node(object):
    def __init__(self,ownGeometry,edge1,edge2):
        self.geom = ownGeometry
        self.edge1 = edge1
        self.edge2 = edge2
    

    
def isolateNetwork(folderPath,siteLayerName,lineLayerName,x,y):
    # Load Lines
    path = folderPath + "\\" + lineLayerName + "\\" + lineLayerName + ".shp"
    
    path_sites = folderPath + "\\" + siteLayerName + "\\" + siteLayerName + ".shp"
    # Buffer around userClick
    print("Now buffer around userClick")
    oRef = osr.SpatialReference()
    oRef.ImportFromEPSG(4326)

    # Reproject
    targRef = osr.SpatialReference()
    targRef.ImportFromEPSG(26918) # NY State Projection; consider a check from the provided site dataset instead
    
    # Create point from x,y and targRef
    ctran = osr.CoordinateTransformation(oRef,targRef)
    [p_lng,p_lat,z] = ctran.TransformPoint(x,y)
    inputPointProj = ogr.Geometry(ogr.wkbPoint)
    inputPointProj.SetPoint_2D(0,p_lng,p_lat)


    print("I got to here")

    geomBuffer = inputPointProj.Buffer(10000) # Buffer 10 km around the geometry
 
    # Load Selected Sites
    sitesDataSource = ogr.Open(path_sites)
    sl = sitesDataSource.GetLayer()
    ctran = osr.CoordinateTransformation(sl.GetSpatialRef(),targRef)
    
    # Intersect of BUFFER and SITES
    interSites = []
    for site in sl:
        ingeom_site = site.GetGeometryRef()
        if ingeom_site.Intersect(geomBuffer):
            # This site is inside the buffer!
            interSites.append((site,ingeom_site.Buffer(1)))
    print("There are {0} sites inside the ring!".format(len(interSites)))
    # Load Selected Lines
    dataSource = ogr.Open(path)
    shpdriver = ogr.GetDriverByName('ESRI Shapefile')
    linesLayer = dataSource.GetLayer()
    
    linesLayer.SetSpatialFilter(geomBuffer)
    
    interLines = []
    # Intersect of BUFFER and LINES
    for line in linesLayer:        
        interLines.append(line)
    print(len(interLines))    
      
    # Buffer all Lines
    lBufferStore = {} # Stores line entry, (buffered geom,Bool flag)
    
    sitesList = [] # Stores Site object (fake site)
    flowlineList = [] # Stores Flowline object (flow)
    
    
    # Create Buffer polygon where user clicked    
    # Find line which ucBuff intersects
    ucBuff = inputPointProj.Buffer(1)
    i = 0
    startingIndex = None
    for line in interLines:       
        e = (line.GetGeometryRef().Buffer(1),False)
        lBufferStore[line] = e # Original Line, Buffered Geometry 
        if ucBuff.Intersects(e[0]):
            startingIndex = line
            print("I will start at line index {0}".format(startingIndex))
        i += 1
        
    if startingIndex is None:
        print("ERROR: USER_CLICK does not intersect any existing lines")
        return
    
    #queue.append(startingIndex)
    
#     queue = [] # Stores keys
#     while len(queue) > 0:
#         e = queue.pop(0) # This will be the line
#         if lBufferStore[e] == True:
#             # We have already visited this
#             continue
#         else:
#             _npt = e.GetPointCount()
#             upBuff = e.GetPoint(0).Buffer(1)
#             downBuff = e.GetPoint(_npt - 1).Buffer(1)
#             # See if any of the sites are also that endpoint
#             b_f = [False,False]
#             for s in interSites:
#                 s_geom = s.GetGeometryRef()
#                 if s_geom.Intersects(upBuff):
#                     # Found existing upper extent
#                     b_f[0] = True
#                 elif s_geom.Intersects(downBuff):
#                     # Found existing lower extent
#                     b_f[1] = True
            
#             # Find out which lines intersect on that point
#             for k,v in lBufferStore.items():
#                 if k == e:
#                     continue
#                 else:
#                     k_geom = k.GetGeometryRef()
#                     if k_geom.Intersects(upBuff) and not b_f[0]:
#                         # Found an upstream flowline
#                     elif k_geom.Intersects(downBuff) and not b_f[1]:
#                         # Found a downstream flowline
                
#             # Declare we have now visited and added this line
#             lBufferStore[e] = True
            
            
        
    
    
    
        
    # Store which ones intersect which other ones
    
    # Create Node objects at the difference of the intersections
    # Flag nodes which share Intersection with existing sites
    
    # Derive Network Relationship

    # Display all in folium GeoJSON
    m = folium.Map(location=[y,x],zoom_start=13)
    folium.Marker([y,x],popup='<b>USER_CLICK</b>').add_to(m)
    
    for i in range(len(interSites)):
        sentry = interSites[i]
        geoJ = geomToGeoJSON(sentry[1],"",10,sl.GetSpatialRef(),oRef)
        if i == 12:
            print(geoJ)
        folium.GeoJson(data=geoJ).add_to(m)  
    
    # Draw all the interLines onto the folium map
    for lentry in interLines:
        
        geoJ = geomToGeoJSON(lentry.GetGeometryRef(),"",10,linesLayer.GetSpatialRef(),oRef)
        folium.GeoJson(data=geoJ).add_to(m)    
    
    display(m)
    
    

In [24]:
isolateNetwork("C:\\Users\\mpanozzo\\Desktop\\GDAL_Data_PR","ProjectedSites","NHDFlowline_Project",-73.939645996,42.378012068)

Now buffer around userClick
I got to here
There are 11 sites inside the ring!
861
I will start at line index <osgeo.ogr.Feature; proxy of <Swig Object of type 'OGRFeatureShadow *' at 0x000001CDE88847B0> >
